In [ ]:
import pickle

class Cell():
    """
    Object representing a cell with walls in its compass directons.
    - get_pos() -> tuple: returns position of cell obj
    - get_neighbors() -> list[tuple]: return all possible unverified neighbors

    """
    def __init__(self, x: int, y: int, hasWalls:bool=True):
        self.x, self.y = x, y
        self.walls={'U': True, 'D': True, 'L': True, 'R': True}
        if not hasWalls: 
            for key in self.walls.keys(): self.walls[key]=False
        self.h = 0

    def get_pos(self) -> tuple:
        return (self.x, self.y)
    
    def get_neighbors(self) -> list[tuple]:
        return [(self.x+dx, self.y+dy) for dy, dx in [(0,-1), (0,1), (-1, 0), (1, 0)]]
    
    def __repr__(self):
        return f"({self.x}, {self.y})"
    
    
    # comparison operator overload for comparing objects
    # tie breaking for part 2 should be implemented here... 
    # analyze tie-breaking for cells with larger g-values vs smaller g-values
    def __lt__(self, other):
        return self.h < other.h
        #return True

    """
    # having the __eq__ comparison removes hash-ability for the cell object when trying to add the object to a set. i do not know why this happens, but it is worth looking into.
    def __eq__(self, other):
        return not (self.f < other.f) and not (other.f < self.f)
    """

class MazeArray():
    """
    Object representing Maze array of Cell() objects
    - cell_at(n: tuple) -> Cell: returns cell object at position n or (x, y)
    - remove_wall(currCell: Cell, otherCell: Cell) -> None: removes the adjacent wall to currCell and otherCell
    - wall_exists(currCell: Cell, otherCell: Cell) -> bool: returns True if a wall exists between currCell and otherCell
    - get_position() -> Cell: returns agent's current position
    - update_position(agent: Cell) -> None: used to update the string representation with the agent's location
    - set_end(end: Cell) -> None: set the end position
    - get_end() -> Cell: get the end position
    - set_boundary_walls() -> None: To prevent no boundary walls. (implemented to avoid red highlights for printing empty row in vscode)
    - update_walls(otherMaze: MazeArray, cell: Cell) -> None: updates cell's walls from another cell's wall attribute. will also update the neighboring cell's wall that are facing the origin cell. 
    - valid_move(nextpos: Cell) -> bool: returns whether a move is valid
    """
    def __init__(self, m: int, n: int, hasWalls=True):
        self.n = n
        self.m = m
        self.totalnodes = (n+1)*(m+1)
        self.world = []
        ctr = 0
        self.agent = (-1, -1)
        self.end = (-1, -1)
        for i in range(m):
            self.world.append(list())
            for j in range(n):
                self.world[i].append(Cell(j, i, hasWalls))
                ctr+=1
        print(f"{ctr} nodes created")

    def cell_at(self, n: tuple) -> Cell:
        return self.world[n[1]][n[0]]
    
    def remove_wall(self, currCell: Cell, otherCell: Cell) -> None:
        if abs(currCell.x-otherCell.x)+abs(currCell.y-otherCell.y) != 1:
            raise Exception("{otherCell} not adjacent to self {currCell}")

        if otherCell.x > currCell.x:
            currCell.walls['R'] = False
            otherCell.walls['L'] = False
        elif otherCell.x < currCell.x:
            currCell.walls['L'] = False
            otherCell.walls['R'] = False
        elif otherCell.y > currCell.y:
            currCell.walls['D'] = False
            otherCell.walls['U'] = False
        elif otherCell.y < currCell.y:
            currCell.walls['U'] = False
            otherCell.walls['D'] = False
    
    def wall_exists(self, currCell: Cell, otherCell: Cell) -> bool:
        if abs(currCell.x-otherCell.x)+abs(currCell.y-otherCell.y) != 1:
            raise Exception("{otherCell} not adjacent to self {currCell}")

        if otherCell.x > currCell.x:
            return currCell.walls['R'] or otherCell.walls['L']
        elif otherCell.x < currCell.x:
            return currCell.walls['L'] or otherCell.walls['R']
        elif otherCell.y > currCell.y:
            return currCell.walls['D'] or otherCell.walls['U']
        elif otherCell.y < currCell.y:
            return currCell.walls['U'] or otherCell.walls['D']
    
    def get_position(self) -> Cell:
        return self.cell_at(self.agent)

    def update_position(self, agent: Cell):
        self.agent = agent.get_pos()

    def set_end(self, end: Cell) -> None:
        self.end = end.get_pos()
    
    def get_end(self):
        return self.cell_at(self.end)
    
    def set_boundary_walls(self) -> None:
        for i in range(self.m):
            self.cell_at((0, i)).walls['L'] = True
            self.cell_at((self.n-1, i)).walls['R'] = True
        for i in range(self.n):
            self.cell_at((i, 0)).walls['U'] = True
            self.cell_at((i, self.m-1)).walls['D'] = True
    
    def update_walls(self, otherMaze, cell: Cell) -> None:
        def isValid(cell: tuple):
            return 0 <= cell[0] < self.n and 0 <= cell[1] < self.m
        
        self.cell_at(cell.get_pos()).walls = cell.walls

        if isValid((cell.x+1, cell.y)): self.cell_at((cell.x+1, cell.y)).walls['L'] = otherMaze.cell_at((cell.x+1, cell.y)).walls['L']
        if isValid((cell.x-1, cell.y)): self.cell_at((cell.x-1, cell.y)).walls['R'] = otherMaze.cell_at((cell.x-1, cell.y)).walls['R']
        if isValid((cell.x, cell.y + 1)): self.cell_at((cell.x, cell.y + 1)).walls['U'] = otherMaze.cell_at((cell.x, cell.y+1)).walls['U']
        if isValid((cell.x, cell.y - 1)): self.cell_at((cell.x, cell.y - 1)).walls['D'] = otherMaze.cell_at((cell.x, cell.y-1)).walls['D']

    def valid_move(self, nextpos: Cell):
        if abs(self.agent[0]-nextpos.x)+abs(self.agent[1]-nextpos.y) != 1:
            raise Exception("Cannot Move: {otherCell} not adjacent to self {currCell}")
        if self.agent[0] > nextpos.x: return not self.cell_at(self.agent).walls['L']
        if self.agent[0] < nextpos.x: return not self.cell_at(self.agent).walls['R']
        if self.agent[1] > nextpos.y: return not self.cell_at(self.agent).walls['U']
        if self.agent[1] < nextpos.y: return not self.cell_at(self.agent).walls['D']

    def __repr__(self):
        return str(self.world)
    
    def __str__(self):
        mystr = ""
        for i in range(self.n):
            mystr += "+--" if self.cell_at((i, 0)).walls['U'] else "+  "
        mystr+='+\n'
        for i in range(self.m):
            horstr = "+"
            verstr = "|" if self.cell_at((0, i)).walls['L'] else " "
            for j in range(self.n):
                if self.cell_at((j,i)).walls['R']: verstr += "  |"
                else: verstr += "   "
                if self.cell_at((j,i)).walls['D']: horstr += "--+"
                else: horstr += "  +"

                # if cell is agent position or end position, then indicate position
                if ((j,i))==self.agent:
                    verstr=verstr[:-3]
                    if self.cell_at((j,i)).walls['R']: verstr += "SS|"
                    else: verstr+="SS "
                elif ((j,i))==self.end:
                    verstr=verstr[:-3]
                    if self.cell_at((j,i)).walls['R']: verstr += "EE|"
                    else: verstr+="EE "

            mystr+=verstr + '\n' + horstr + '\n'
        return mystr

def read_maze(file) -> MazeArray:
    with open(file, 'rb') as f:
        return pickle.load(f)

def write_maze(maze: MazeArray, file) -> None:
    with open(file, 'wb') as f:
        pickle.dump(maze, f)

In [ ]:

from collections import deque
import random
from functools import lru_cache

def create_maze(m=20, n=20, isRandom=False, hasWalls=True):
    if m < 1 or n < 1:
        raise Exception(f"Invalid Maze Size: {m}x{n}")

    if isRandom:
        m = random.randrange(1,m+1)
        n = random.randrange(1,n+1)
        
    return MazeArray(m, n, hasWalls)
    
# iterative implementation of DFS
def dfs(maze: MazeArray, *args):
    # initializing variables
    m, n = maze.m, maze.n
    visited: set(Cell) = set()

    # SUB METHODS START ------------------------------------------------------------------------
    # checks if starting DFS position has been provided, else, provides random position pos
    def check_args(args: list[int]) -> Cell: 
        if not args:
            pos = (random.randrange(n), random.randrange(m))
        elif len(args)==2:
            pos = (args[0], args[1])
        else:
            raise Exception(f"{len(args)} arguments given but only 0 or 2 accepted.")
        return maze.cell_at((pos[0], pos[1]))

    # a cell is valid when it is inside the grid and hasn't been visited
    def is_valid(cell: Cell | tuple) -> bool:
        if isinstance(cell, Cell):
            return 0 <= cell.x < n and 0 <= cell.y < m and cell not in visited
        elif isinstance(cell, tuple):
            return 0 <= cell[0] < n and 0 <= cell[1] < m and maze.cell_at((cell[0], cell[1])) not in visited

    # fills neighbors with valid maze Cells

    def get_valid_neighbors(cell: Cell) -> list[Cell]:
        unverified_neighbors = cell.get_neighbors()
        for i in range(4): # for four possible neighbors
            if is_valid(unverified_neighbors[i]):
                unverified_neighbors[i] = maze.cell_at((unverified_neighbors[i][0], unverified_neighbors[i][1]))
        return list(filter(lambda x: isinstance(x, Cell), unverified_neighbors)) # returns the list of validated neighbors
        
    # SUB METHODS END ---------------------------------------------------------------------------
    # adding in start position
    cell = check_args(args)
    visited.add(cell)
    stack: list[Cell] = [cell]

    while stack:
        visited.add(cell)
        neighbors = get_valid_neighbors(cell)
        if not neighbors:
            cell = stack.pop()
            continue
        nextCell = random.choice(neighbors)
        maze.remove_wall(cell, nextCell)
        stack.append(cell)
        cell = nextCell

def create_position(maze: MazeArray) -> Cell:
    return maze.cell_at((random.randrange(maze.n), random.randrange(maze.m)))

In [ ]:

# Binary Min Heap
def sift(ls, node):
    left = node * 2 + 1
    right = node * 2 + 2
    has_right = right < len(ls)

    min_index = left

    if has_right and ls[right] < ls[left]:
        min_index = right

    if ls[min_index] < ls[node]:
        ls[min_index], ls[node] = ls[node], ls[min_index]
        node = min_index

    return node  # return where the original parent now resides


def heapify(ls):
    if len(ls) < 2:
        return

    current = int((len(ls) - 2) / 2)

    while current >= 0:
        sift(ls, current)
        current -= 1

def heappush(ls, value):
    index = len(ls)
    ls.append(value)
    parent = int((index - 1) / 2)

    while ls[index] < ls[parent]:
        ls[index], ls[parent] = ls[parent], ls[index]
        index = parent
        parent = int((index - 1) / 2)


def heappop(ls):
    if len(ls) == 1:
        return ls.pop()

    item = ls[0]
    ls[0] = ls.pop()

    index = -1
    next_i = 0

    while next_i != index and int(next_i * 2 + 1) < len(ls):
        index = next_i
        next_i = sift(ls, index)
    return item



In [ ]:
import numpy as np
import random
from matplotlib.colors import ListedColormap
import matplotlib.colors as pltColors
import matplotlib.pyplot as plt
import matplotlib 
import BinaryHeap as bh
import time 
import sys
from mazeGen import generateMazes

############## Part 1 
def isDeadEnd(y,x,visited) :     
	for i in range(-1,1):   #i in -1,0,1
		for j in range(-1,1):#i in -1,0,1
			if(not(i==0 and j==0)):#as if i==0 and j==0 then we are in teh same cell 	 
				if(x+i>=0 and x+i< num_cols ):
					if( y+j >=0 and y+j < num_rows):
						if((x+i,y+j) not in visited):
							return False,y+j,x+i #There's an unvisited neighbour 
	return True,-1,-1;                           #There's no unvisited neighbour
   
def isValidRow(y):
	if(y>=0 and y<num_rows):
		return True; 
	return False; 

def isValidCol(x):
	if(x>=0 and x<num_cols):
		return True; 
	return False; 
	
def generateMazes(mazesNumber,num_rows,num_cols):

	#Generate the 50 mazes 
	##########################################################
	# initially set all of the cells as unvisited
	maze      = np.zeros((mazesNumber,num_rows,num_cols))


	for mazeInd in range(0,mazesNumber) :
		print("Generate Maze : " + str(mazeInd+1));
		visited = set()  # Set for visitied nodes 
		stack   = []     # Stack is empty at first 

		##########################################################
		#start from a random cell	
		row_index = random.randint(0,num_rows-1)#Must choose valid row index 
		col_index = random.randint(0,num_cols-1)#Must choose valid col index 
		#mark it as visitied 	
		print("_______________ Start ________________\n")
		print("Loc["+str(row_index)+"],["+str(col_index)+"] = 1")
		visited.add((row_index , col_index))       #Visited 
		maze [mazeInd , row_index , col_index] = 1 #Unblocked 
		

		##########################################################
		#Select a random neighbouring cell to visit that has not yet been visited. 
		print("\n\n_______________ DFS ________________\n")
		while(len(visited) < num_cols*num_rows): #Repeat till visit all cells 
		
			crnt_row_index = row_index+random.randint(-1,1)#neighbor
			crnt_col_index = col_index+random.randint(-1,1)#neighbor
			i=0;isDead=False;
			while ((not isValidRow(crnt_row_index)) or (not isValidCol(crnt_col_index) )or ((crnt_row_index,crnt_col_index) in visited) ):
				# no need to write also "or (crnt_row_index==row_index and crnt_col_index==col_index)" as if this happened then it would be visited 
				crnt_row_index = row_index+random.randint(-1,1)
				crnt_col_index = col_index+random.randint(-1,1)
				i = i+1
				#print("dtuck"+str(i))
				if(i==8):
					#Reach dead end 
					isDead = True
					break
			if(not isDead):
				visited.add((crnt_row_index , crnt_col_index)) 
			
			rand_num  = random.uniform(0, 1)

			if( rand_num < 0.3 and not isDead) : 
				# With 30% probability mark it as blocked. 
				maze [mazeInd , crnt_row_index , crnt_col_index] = 0 #Leave the block  
				print("Loc["+str(crnt_row_index)+"],["+str(crnt_col_index)+"] = 0")				
				#to start get the neighbors of this cell next time 
				row_index = crnt_row_index
				col_index = crnt_col_index
			else : 
				if(not isDead):
					# With 70% mark it as unblocked and in this case add it to the stack.
					maze [mazeInd , crnt_row_index , crnt_col_index] = 1 #Unblocked 
					print("Loc["+str(crnt_row_index)+"],["+str(crnt_col_index)+"] = 1")				
					stack.append((crnt_row_index,crnt_col_index))
					isDead,unvisitRow , unvisitCol = isDeadEnd(row_index,col_index,visited)
				if(isDead == True):#if no unvisited neighbour 
					#backtrack to parent nodes on the search tree until it reaches a cell with an unvisited neighbour
					while(len(stack)>0):
						parent_row,parent_col = stack.pop();
						isDead,unvisitRow , unvisitCol = isDeadEnd(parent_row,parent_col,visited)
						if(isDead == False):
							break;
					# Now wither we reach not dead end or stack is empty 
					if(len(stack)>0):
						visited.add((unvisitRow,unvisitCol))
						row_index = unvisitRow
						col_index = unvisitCol
					else :
						#Repeat the whole process from a point not vistited
						row_index = random.randint(0,num_rows-1)
						col_index = random.randint(0,num_cols-1)
						if(len(visited)< num_cols*num_rows):
							while ( (not isValidRow(row_index)) or (not isValidCol(col_index)) or ((row_index,col_index) in visited) ):
								row_index = random.randint(0,num_rows-1)
								col_index = random.randint(0,num_cols-1)
								#print(str(row_index)+","+str(col_index))
						#mark it as visitied 	
						visited.add((row_index , col_index))       #Visited 					
				else : #No dead Node 
					visited.add((unvisitRow,unvisitCol))
					row_index = unvisitRow
					col_index = unvisitCol

				
	return maze
	
def poltline(A,col):
    x, y = zip(*A)
    temp, = plt.plot(y,x,color=col)
    print("Line type ")
    print(temp.__class__.__name__)
    return temp
	

# This method is to get Manhattan Distances for A*
def ManhattanDistanes(curr, goal):
    return abs(curr[0] - goal[0]) + abs(curr[1] - goal[1])

# This method will find the path for Forward A*
def getForwardPath(dict, curr, goal):
    temp = [goal]
    next = dict[goal]
    while next != curr:
        temp.insert(0,next)
        next = dict[next]
    temp.insert(0,next)
    return temp

# This method will find the path for the Backward A*
def getBackwardPath(dict, curr, goal):
    temp = [curr]
    next = dict[curr]
    while next != goal:
        temp.append(next)
        next = dict[next]
    temp.append(next)
    return temp

# This method will find the neighbors of the current cell
def nextNeighbor (cellNumber, length):
    neighborList = [(cellNumber[0],cellNumber[1]+1),(cellNumber[0],cellNumber[1]-1),(cellNumber[0]+1,cellNumber[1]),(cellNumber[0]-1,cellNumber[1])]
    temp = []
    for x in neighborList:
        if -1 < x[0] < length and -1 < x[0] < length:
            temp.append(x)
    return temp

# This method will update the neighbors to empty and blocked
def update(map,length,curr,bList):
    goodCells = nextNeighbor(curr,length)
    for x in goodCells:
        print(x)
        print(map[2][x])
		
        if x in bList:
            map[2][x] = 2 # It's blocked
        else:
            map[2][x] = 1 # It's empty


# This method will display the intial map

def dispMap(length, bList = None, pList = None, map = None, old = None):
    if not(map is None):
        d = map[2]
        f, a = plt.subplots()
        print(d)
        a.imshow(d)#, ListedColormap(['grey','white','black']))
    else:
        d = np.zeros((length, length))
        if not (bList is None):
            x = bList
        f, a = plt.subplots()
        a.imshow(d)#,  ListedColormap(['white','black']))
    print(a)
    #a.set_xticks(np.arange(( -0.5), length, 1))#, minor = True)
    #a.set_yticks(np.arange(( -0.5), length, 1))#, minor=True)
    a.grid(which= 'minor', color='black', linestyle='-', linewidth=1)
    
    if pList != None:
        poltline(pList, 'red')
    if old != None:
        poltline(old, 'green')
    plt.show()


def showLaunch(length, bList = None, pList = None, map = None, old = None): 
	#This method will add the colors to the maze 
    d = map[2]
    f,a = plt.subplots()
    colorMap = matplotlib.colors.ListedColormap(['grey','white','black'])
    boundaries = [-0.5,0.5,1.5,2.5]
    n = matplotlib.colors.BoundaryNorm(boundaries, colorMap.N)
    image = a.imshow(d,interpolation='nearest', origin='lower')
    a.set_xticks(np.arange(-.5, length, 1), minor=True)
    a.set_yticks(np.arange(-.5, length, 1), minor=True)
    a.grid(which= 'minor', color='black', linestyle='-', linewidth=1)
    oLine = poltline(old,'green')
    pLine = poltline(pList, 'red')
    plt.pause(1)
    return image, f, oLine, pLine

def showUpdate(image, f, oLine, pLine, map, aList, bList):
	# This method will add the path line to the maze 
    image.set_data(map)
    x,y = zip(*aList)
    oLine.set_xdata(y)
    oLine.set_ydata(x)
    x,y = zip(*bList)
    pLine.set_xdata(y)
    pLine.set_ydata(x)
    plt.pause(1)
    f.canvas.draw()

def adaptedAStar(mLength,bList, start, goal, show = False):
	# Here we implement Adaptive A* which will take the length, start, and goal as inputs
	maximumG = mLength * mLength
	currCell = start 
	gCell = goal
	magMap = np.zeros((5,mLength, mLength))
	magMap[3] = np.zeros((mLength,mLength), dtype = bool)
	magMap[4] = np.zeros((mLength,mLength), dtype = bool)
	magMap[2][currCell] = 1
	maxG = 0 
	totalSteps = 0 
	totalExpense = 0 
	currTrack = [start]
	update(magMap,mLength,currCell,bList)
	while currCell != gCell:
		totalSteps += 1 
		magMap[1][currCell] = 0
		magMap[0][currCell] = totalSteps
		magMap[1][gCell] = np.inf
		magMap[0][gCell] = totalSteps
		oList = []
		oDict = dict()
		fDict = dict()
		magMap[4] = np.zeros((mLength,mLength), dtype = bool)
		bh.insert(oList, oDict, ManhattanDistanes(currCell,gCell), currCell)
		while oList and magMap[1][gCell] > oList[0]:
			coloredCell = bh.pop(oList, oDict)
			magMap[4][coloredCell] = True
			totalExpense += 1
			for newCell in nextNeighbor(coloredCell,mLength):
				if magMap[2][newCell] != 2 :
					if magMap[3][newCell]:
						newHeuris = maxG - magMap[1][newCell]
					else:
						newHeuris = ManhattanDistanes(newCell,gCell)
					if magMap[0][newCell] < totalSteps:
						magMap[1][newCell] = np.inf
						magMap[0][newCell] = totalSteps
					if magMap[1][newCell] > magMap[1][coloredCell] + 1:
						fDict[newCell] = coloredCell
						newG = magMap[1][coloredCell] + 1
						bh.insert(oList, oDict, (maximumG*( newG +  newHeuris) -  newG) , newCell )
						magMap[1][newCell] = newG
		magMap[3] = magMap[4]
		maxG = magMap[1][gCell]
		if not oList:
			return None, None
		currPath = getForwardPath(fDict,currCell,gCell)
		if show:
			if currCell == start:
				plt.ion()
				im, fig,oLine,pLine = showLaunch(mLength, map = magMap, pList = currPath , old = currTrack)
				plt.show()
			else:
				showUpdate(im, fig, oLine,pLine, magMap[2],  currTrack ,currPath)
		for cell in currPath:
			if cell == currCell:
				continue
			else:
				if magMap[2][cell] != 2 :
					currTrack.append(cell)
					currCell = cell
					update( magMap,mLength,currCell,bList)
				else:
					break 
	if show:
		showUpdate(im, fig, oLine,pLine, magMap[2],  currTrack ,currPath)
	return currTrack,totalExpense

def backwardAStarTie(mLength,bList, start, goal, show = False):
	# Here we implement Backward A* which will take the length, start, and goal as inputs
	totalExpense = 0 
	maxG = mLength*mLength
	currCell = start 
	gCell = goal
	magMap = np.zeros((4,mLength, mLength))
	magMap[2][currCell] = 1
	totalSteps = 0 
	currTrack = [start]
	update(magMap,mLength,currCell,bList)
	while currCell != gCell:
		magMap[3] = np.zeros((mLength,mLength))
		totalSteps += 1 
		magMap[1][currCell] = np.inf
		magMap[0][currCell] = totalSteps
		magMap[1][gCell] = 0
		magMap[0][gCell] = totalSteps
		oList = []
		oDict = dict()
		fDict = dict()
		bh.insert(oList, oDict, ManhattanDistanes(currCell,gCell), gCell)
		while oList and magMap[1][currCell] > oList[0]:
			coloredCell = bh.pop(oList, oDict)
			magMap[3][coloredCell] = 1
			totalExpense += 1
			for newCell in nextNeighbor(coloredCell,mLength):
				if magMap[2][newCell] != 2 :
					if magMap[0][newCell] < totalSteps:
						magMap[1][newCell] = np.inf
						magMap[0][newCell] = totalSteps
					if magMap[1][newCell] > magMap[1][coloredCell] + 1:
						magMap[1][newCell] = magMap[1][coloredCell] + 1
						fDict[newCell] = coloredCell
						bh.insert(oList, oDict, maxG*(magMap[1][newCell] + ManhattanDistanes(newCell,currCell)) - magMap[1][newCell] , newCell )
		if not oList:
			return None, None
		
		if show:
			currPath = getBackwardPath(fDict,currCell,gCell)
			if currCell == start:
				plt.ion()
				im, fig,oLine,pLine = showLaunch(mLength, map = magMap, pList = currPath , old = currTrack)
				plt.show()
			else:
				showUpdate(im, fig, oLine,pLine, magMap[2],  currTrack ,currPath)
		while currCell != gCell:
			cell = fDict[currCell]
			if magMap[2][cell] != 2 :
				currTrack.append(cell)
				currCell = cell
				update( magMap,mLength,currCell,bList)
			else:
				break
	if show:
		showUpdate(im, fig, oLine,pLine, magMap[2],  currTrack ,currPath) 
	return currTrack, totalExpense

def forwardAStarTie(mLength,bList, start, goal, show = False):
	# Here we implement Forward A* which will take the length, start, and goal as inputs
	totalExpense = 0
	maxG = mLength* mLength
	currCell = start 
	gCell = goal
	magMap = np.zeros((4,mLength, mLength))
	magMap[2][currCell] = 1
	totalSteps = 0 
	currTrack = [start]
	update(magMap,mLength,currCell,bList)
	while currCell != gCell:
		totalSteps += 1 
		magMap[1][currCell] = 0
		magMap[0][currCell] = totalSteps
		magMap[1][gCell] = np.inf
		magMap[0][gCell] = totalSteps
		oList = []
		oDict = dict()
		fDict = dict()
		bh.insert(oList, oDict, ManhattanDistanes(currCell,gCell), currCell)
		while oList and magMap[1][gCell] > oList[0]:
			coloredCell = bh.pop(oList, oDict)
			magMap[3][coloredCell] = 1
			totalExpense += 1
			for newCell in nextNeighbor(coloredCell,mLength):
				if  magMap[2][newCell] != 2 :
					if magMap[0][newCell] < totalSteps:
						magMap[1][newCell] = np.inf
						magMap[0][newCell] = totalSteps
					if magMap[1][newCell] > magMap[1][coloredCell] + 1:
						magMap[1][newCell] = magMap[1][coloredCell] + 1
						fDict[newCell] = coloredCell
						bh.insert (oList, oDict, ( maxG*(magMap[1][newCell] + ManhattanDistanes(newCell,gCell)) - magMap[1][newCell] )  , newCell ) 
		if not oList:
			return None, None
		currPath = getForwardPath(fDict,currCell,gCell)
		if show:
			if currCell == start:
				plt.ion()
				im, fig,oLine,pLine = showLaunch(mLength, map = magMap, pList = currPath , old = currTrack)
				plt.show()
			else:
				showUpdate(im, fig, oLine,pLine, magMap[2],  currTrack ,currPath)
		for cell in currPath:
			if cell == currCell:
				continue
			else:
				if magMap[2][cell] != 2 :
					currTrack.append(cell)
					currCell = cell
					update( magMap,mLength,currCell,bList)
				else:
					break 
	if show:
		showUpdate(im, fig, oLine,pLine, magMap[2],  currTrack ,currPath)
	return currTrack, totalExpense

def repeatedForwardAStar(mLength,bList, start, goal, show = False):
	# Here we implement Repeated Forward A* which will take the length, start, and goal as inputs
	totalExpense = 0
	maxG = mLength* mLength
	currCell = start 
	gCell = goal
	magMap = np.zeros((4,mLength, mLength))
	magMap[2][currCell] = 1
	totalSteps = 0 
	currTrack = [start]
	update(magMap,mLength,currCell,bList)
	while currCell != gCell:
		totalSteps += 1 
		magMap[1][currCell] = 0
		magMap[0][currCell] = totalSteps
		magMap[1][gCell] = np.inf
		magMap[0][gCell] = totalSteps
		oList = []
		oDict = dict()
		fDict = dict()
		bh.insert(oList, oDict, ManhattanDistanes(currCell,gCell), currCell)
		while oList and magMap[1][gCell] > oList[0]:
			coloredCell = bh.pop(oList, oDict)
			totalExpense += 1
			for newCell in nextNeighbor(coloredCell,mLength):
				if  magMap[2][newCell] != 2 :
					if magMap[0][newCell] < totalSteps:
						magMap[1][newCell] = np.inf
						magMap[0][newCell] = totalSteps
					if magMap[1][newCell] > magMap[1][coloredCell] + 1:
						magMap[1][newCell] = magMap[1][coloredCell] + 1
						fDict[newCell] = coloredCell
						bh.insert (oList, oDict, ( maxG*(magMap[1][newCell] + ManhattanDistanes(newCell,gCell)) + magMap[1][newCell] )  , newCell ) 
		if not oList:
			return None, None
		currPath = getForwardPath(fDict,currCell,gCell)
		if show:
			if currCell == start:
				plt.ion()
				im, fig,oLine,pLine = showLaunch(mLength, map = magMap, pList = currPath , old = currTrack)
				plt.show()
			else:
				showUpdate(im, fig, oLine,pLine, magMap[2],  currTrack ,currPath)
		for cell in currPath:
			if cell == currCell:
				continue
			else:
				if magMap[2][cell] != 2 :
					currTrack.append(cell)
					currCell = cell
					update( magMap,mLength,currCell,bList)
				else:
					break 
	if show:
		showUpdate(im, fig, oLine,pLine, magMap[2],  currTrack ,currPath)
	return currTrack, totalExpense

def repeatedForwardAStarB(mLength,bList, start, goal, show = False):
	# Here we implement Repeated Forward A* which will take the length, start, and goal as inputs
	totalExpense = 0
	currCell = start 
	gCell = goal
	magMap = np.zeros((3,mLength, mLength))
	magMap[2][currCell] = 1
	totalSteps = 0 
	currTrack = [start]
	update(magMap,mLength,currCell,bList)
	while currCell != gCell:
		totalSteps += 1 
		magMap[1][currCell] = 0
		magMap[0][currCell] = totalSteps
		magMap[1][gCell] = np.inf
		magMap[0][gCell] = totalSteps
		oList = []
		oDict = dict()
		fDict = dict()
		bh.insert(oList, oDict, ManhattanDistanes(currCell,gCell), currCell)
		while len(oList)>0 and magMap[1][gCell] > oList[0]:
			coloredCell = bh.pop(oList, oDict)
			totalExpense +=1
			for newCell in nextNeighbor(coloredCell,mLength):
				if  magMap[2][newCell] != 2 :
					if magMap[0][newCell] < totalSteps:
						magMap[1][newCell] = np.inf
						magMap[0][newCell] = totalSteps
					if magMap[1][newCell] > magMap[1][coloredCell] + 1:
						magMap[1][newCell] = magMap[1][coloredCell] + 1
						fDict[newCell] = coloredCell
						bh.insert(oList, oDict, (magMap[1][newCell] + ManhattanDistanes(newCell,gCell)) , newCell )
		if not oList:
			return None, None
		currPath = getForwardPath(fDict,currCell,gCell)
		if show:
			
			if currCell == start:
				plt.ion()
				im, fig,oLine,pLine = showLaunch(mLength, map = magMap, pList = currPath , old = currTrack)
				plt.show()
			else:
				showUpdate(im, fig, oLine,pLine, magMap[2],  currTrack ,currPath)
		for cell in currPath:
			if cell == currCell:
				continue
			else:
				if magMap[2][cell] != 2 :
					currTrack.append(cell)
					currCell = cell
					update( magMap,mLength,currCell,bList)
				else:
					break 
	if show:
		showUpdate(im, fig, oLine,pLine, magMap[2],  currTrack ,currPath)
	return currTrack,totalExpense

def repeatedBackwardAStar(mLength,bList, start, goal, show = False):
	# Here we implement Repeated Backward A* which will take the length, start, and goal as inputs
	totalExpense = 0
	currCell = start 
	gCell = goal
	magMap = np.zeros((3,mLength, mLength))
	magMap[2][currCell] = 1
	totalSteps = 0 
	currTrack = [start]
	update(magMap,mLength,currCell,bList)
	while currCell != gCell:
		totalSteps += 1 
		magMap[1][currCell] = np.inf
		print("magMap")
		print(magMap [1,1,2])
		print("currCell")
		print( currCell )
		
		
		magMap[0][currCell] = totalSteps
		magMap[1][gCell] = 0
		magMap[0][gCell] = totalSteps
		oList = []
		oDict = dict()
		fDict = dict()
		bh.insert(oList, oDict, ManhattanDistanes(currCell,gCell), gCell)
		print("olist")
		print(oList)
		while oList :
			print(magMap[1][currCell])
			
			if(magMap[1][currCell] <= oList[0]):
				break;
			totalExpense += 1
			coloredCell = bh.pop(oList, oDict)
			for newCell in nextNeighbor(coloredCell,mLength):
				if magMap[2][newCell] != 2 :
					if magMap[0][newCell] < totalSteps:
						magMap[1][newCell] = np.inf
						magMap[0][newCell] = totalSteps
					if magMap[1][newCell] > magMap[1][coloredCell] + 1:
						magMap[1][newCell] = magMap[1][coloredCell] + 1
						fDict[newCell] = coloredCell
						bh.insert(oList, oDict,magMap[1][newCell] + ManhattanDistanes(newCell,currCell), newCell )
		if not oList:
			return None, None
		
		if show:
			currPath = getBackwardPath(fDict,currCell,gCell)
			if currCell == start:
				plt.ion()
				im, fig,oLine,pLine = showLaunch(mLength, map = magMap, pList = currPath , old = currTrack)
				plt.show()
			else:
				showUpdate(im, fig, oLine,pLine, magMap[2],  currTrack ,currPath)
		while currCell != gCell:
			cell = fDict[currCell]
			if magMap[2][cell] != 2 :
				currTrack.append(cell)
				currCell = cell
				update( magMap,mLength,currCell,bList)
			else:
				break 
	if show:
		showUpdate(im, fig, oLine,pLine, magMap[2],  currTrack ,currPath)
	return currTrack, totalExpense




if __name__ == '__main__':

	#####################################################
	# Generate mazes 101*101
	num_rows    = 101
	num_cols    = 101
	mazesNumber = 50
	#mazes = generateMazes(mazesNumber,num_rows,num_cols)
	#3D numpy array for the 50 mazes 
	

	#####################################################
	# Generate mazes 5*5
	num_rows    = 5
	num_cols    = 5
	mazesNumber = 3
	mazes2 = generateMazes(mazesNumber,num_rows,num_cols)

	mLength = 5;
	bList = [(2,3),(3,4),(3,3),(4,3),(4,4),(5,4)];
	start = (1,1);
	goal  = (3,2);
	
	#np.savetxt('maze '+str(mazeInd)+'.txt',mazes[mazeInd].astype(int) ,fmt='%i', delimiter=",");

	length= 5            ;
	bList = [(2,3),(3,4),(3,3),(4,3),(4,4),(5,4)]; # Example given in assignment
	pList = [(3,1),(3,4)];
	map   = mazes2     ;
	old   = None         ;
	dispMap(list, bList  , pList  , map  , old  );
	
	startX =input("Please , Enter valid x coordinate for the start point");
	startY =input("Please , Enter valid y coordinate for the start point");
	
	goalX  =input("Please , Enter valid x coordinate for the goal point"); 
	goalY  =input("Please , Enter valid y coordinate for the goal point"); 
	

	#####################################################
	# Repeated Forward A*
	print("Repeated Forward A*")
	repeatedForwardAStarB(mLength,bList, start, goal, True);
	print("____________________________________")

	#####################################################
	# Repeated Backward A*
	print("Repeated Backward A*")
	repeatedBackwardAStar(mLength,bList, start, goal, True);
	print("____________________________________")

	#####################################################
	# Repeated forward A with smaller g*
	print("Repeated Forward A* with smaller g-values")
	repeatedForwardAStar(mLength,bList, start, goal, True);
	print("____________________________________")


	#####################################################
	# Adaptive A*
	print("Adapted A*")
	adaptedAStar(mLength,bList, start, goal, True);
		
	#####################################################
	# forward A large g* 

	print("Repeated Forward A* with large g-values")
	forwardAStarTie(mLength,bList, start, goal, True);
	print("____________________________________")

	#####################################################
	# backward A large g* 

	print("Repeated Backward A* with large g-values")		
	backwardAStarTie(mLength,bList, start, goal, True);
	print("____________________________________")
	
	